## Import necessary python packages

In [6]:
import polars as pl
from neo4j import GraphDatabase

## Data management related functions

In [81]:
def get_users_df(file_name: str) -> pl.DataFrame:
    authors = (
        pl.read_csv(f"data/{file_name}.csv")
        .with_columns(
            pl.when(~pl.col("acct").str.contains("@"))
            .then(pl.col("acct") + "@mastodon.social")
            .otherwise(pl.col("acct"))
        )
        .select("id", pl.col("acct").alias("account"))
    )
    return authors

In [82]:
def get_nodes() -> pl.DataFrame:
    nodes = pl.concat([get_users_df("authors"), get_users_df("rebloggers")])
    return nodes

In [83]:
def get_relationships() -> pl.DataFrame:
    relationships = (
        pl.read_csv("data/rebloggers.csv")
        .select("id", "author_id")
        .group_by(["id", "author_id"])
        .len(name="weight")
        .sort("weight", descending=True)
        .select(pl.col("id").alias("reblogger_id"), "author_id", "weight")
    )
    return relationships

In [84]:
nodes = get_nodes()
relationships = get_relationships()

nodes.write_csv("nodes.csv")
relationships.write_csv("relationships.csv")
#!cp data/authors_03_04_10_2025.csv neo4j_db/import/authors_2.csv

In [ ]:
len(relationships)

## Database related functions

In [16]:
def get_database_driver():
    uri = "bolt://localhost:7687"
    return GraphDatabase.driver(uri, auth=("neo4j", "my_password"))

In [75]:
def create_nodes() -> None:
    with get_database_driver().session() as session:
        session.run(
            "CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE"
        )
        session.run(
            """
            LOAD CSV WITH HEADERS FROM 'file:///nodes.csv' AS row
            CALL(row) {
                MERGE (u:User {userId: toInteger(row.id)})
                SET u.account = row.account
            } IN TRANSACTIONS
            """
        )

In [76]:
def create_relationships() -> None:
    with get_database_driver().session() as session:
        session.run(
            """
            LOAD CSV WITH HEADERS FROM 'file:///relationships.csv' AS row
            CALL (row){
              MATCH (r:User {userId: toInteger(row.reblogger_id) })
              MATCH (a:User {userId: toInteger(row.author_id) })
              MERGE (r)-[:REBLOGS {weight: toInteger(row.weight)}]->(a)
            } IN TRANSACTIONS
            """
        )

In [49]:
def delete_all_nodes_and_relationships() -> None:
    with get_database_driver().session() as session:
        session.run("MATCH (n) DETACH DELETE n")

### Execute database releated functions

In [87]:
# Uncomment the below lines to execute the functions you want.
# create_nodes()
create_relationships()
# delete_all_nodes_and_relationships()